In [2]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import key_dict

#gmaps key 
g_key = key_dict['g_key']

 Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [6]:
weather_csv_file = "../WeatherPy/output/weather_info_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,80.13,71,0,7.54,PF,1616357944
1,Ushuaia,-54.8000,-68.3000,55.40,58,40,18.41,AR,1616357707
2,Busselton,-33.6500,115.3333,64.99,95,80,3.00,AU,1616357988
3,Sakakah,29.9697,40.2064,73.40,17,0,4.61,SA,1616358327
4,Saldanha,-33.0117,17.9442,62.60,67,0,6.91,ZA,1616358034


 Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

In [15]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [16]:
new_types_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
